In [195]:
import nest
import os
import json

In [59]:
import urllib
import urllib2
import sys

from optparse import OptionParser

try:
   import json
except ImportError:
   try:
       import simplejson as json
   except ImportError:
       print "No json library available. I recommend installing either python-json"
       print "or simpejson."
       sys.exit(-1)

class Nest:
    def __init__(self, username, password, serial=None, index=0, units="F"):
        self.username = username
        self.password = password
        self.serial = serial
        self.units = units
        self.index = index

    def loads(self, res):
        if hasattr(json, "loads"):
            res = json.loads(res)
        else:
            res = json.read(res)
        return res

    def login(self):
        data = urllib.urlencode({"username": self.username, "password": self.password})

        req = urllib2.Request("https://home.nest.com/user/login",
                              data,
                              {"user-agent":"Nest/1.1.0.10 CFNetwork/548.0.4"})

        res = urllib2.urlopen(req).read()

        res = self.loads(res)

        self.transport_url = res["urls"]["transport_url"]
        self.access_token = res["access_token"]
        self.userid = res["userid"]

    def get_status(self):
        req = urllib2.Request(self.transport_url + "/v2/mobile/user." + self.userid,
                              headers={"user-agent":"Nest/1.1.0.10 CFNetwork/548.0.4",
                                       "Authorization":"Basic " + self.access_token,
                                       "X-nl-user-id": self.userid,
                                       "X-nl-protocol-version": "1"})

        res = urllib2.urlopen(req).read()

        res = self.loads(res)

        self.structure_id = res["structure"].keys()[0]

        if (self.serial is None):
            self.device_id = res["structure"][self.structure_id]["devices"][self.index]
            self.serial = self.device_id.split(".")[1]

        self.status = res

        #print "res.keys", res.keys()
        #print "res[structure][structure_id].keys", res["structure"][self.structure_id].keys()
        #print "res[device].keys", res["device"].keys()
        #print "res[device][serial].keys", res["device"][self.serial].keys()
        #print "res[shared][serial].keys", res["shared"][self.serial].keys()

    def temp_in(self, temp):
        if (self.units == "F"):
            return (temp - 32.0) / 1.8
        else:
            return temp

    def temp_out(self, temp):
        if (self.units == "F"):
            return temp*1.8 + 32.0
        else:
            return temp

    def show_status(self):
        shared = self.status["shared"][self.serial]
        device = self.status["device"][self.serial]

        allvars = shared
        allvars.update(device)

        for k in sorted(allvars.keys()):
             print k + "."*(32-len(k)) + ":", allvars[k]

    def show_curtemp(self):
        temp = self.status["shared"][self.serial]["current_temperature"]
        temp = self.temp_out(temp)

        print "%0.1f" % temp

    def set_temperature(self, temp):
        temp = self.temp_in(temp)

        data = '{"target_change_pending":true,"target_temperature":' + '%0.1f' % temp + '}'
        req = urllib2.Request(self.transport_url + "/v2/put/shared." + self.serial,
                              data,
                              {"user-agent":"Nest/1.1.0.10 CFNetwork/548.0.4",
                               "Authorization":"Basic " + self.access_token,
                               "X-nl-protocol-version": "1"})

        res = urllib2.urlopen(req).read()

        print res

    def set_fan(self, state):
        data = '{"fan_mode":"' + str(state) + '"}'
        req = urllib2.Request(self.transport_url + "/v2/put/device." + self.serial,
                              data,
                              {"user-agent":"Nest/1.1.0.10 CFNetwork/548.0.4",
                               "Authorization":"Basic " + self.access_token,
                               "X-nl-protocol-version": "1"})

        res = urllib2.urlopen(req).read()

        print res

def create_parser():
   parser = OptionParser(usage="nest [options] command [command_options] [command_args]",
        description="Commands: fan temp",
        version="unknown")

   parser.add_option("-u", "--user", dest="user",
                     help="username for nest.com", metavar="USER", default=None)

   parser.add_option("-p", "--password", dest="password",
                     help="password for nest.com", metavar="PASSWORD", default=None)

   parser.add_option("-c", "--celsius", dest="celsius", action="store_true", default=False,
                     help="use celsius instead of farenheit")

   parser.add_option("-s", "--serial", dest="serial", default=None,
                     help="optional, specify serial number of nest thermostat to talk to")

   parser.add_option("-i", "--index", dest="index", default=0, type="int",
                     help="optional, specify index number of nest to talk to")


   return parser

def help():
    print "syntax: nest [options] command [command_args]"
    print "options:"
    print "   --user <username>      ... username on nest.com"
    print "   --password <password>  ... password on nest.com"
    print "   --celsius              ... use celsius (the default is farenheit)"
    print "   --serial <number>      ... optional, specify serial number of nest to use"
    print "   --index <number>       ... optional, 0-based index of nest"
    print "                                (use --serial or --index, but not both)"
    print
    print "commands: temp, fan, show, curtemp, curhumid"
    print "    temp <temperature>    ... set target temperature"
    print "    fan [auto|on]         ... set fan state"
    print "    show                  ... show everything"
    print "    curtemp               ... print current temperature"
    print "    curhumid              ... print current humidity"
    print
    print "examples:"
    print "    nest.py --user joe@user.com --password swordfish temp 73"
    print "    nest.py --user joe@user.com --password swordfish fan auto"

def main():
    parser = create_parser()
    (opts, args) = parser.parse_args()

    if (len(args)==0) or (args[0]=="help"):
        help()
        sys.exit(-1)

    if (not opts.user) or (not opts.password):
        print "how about specifying a --user and --password option next time?"
        sys.exit(-1)

    if opts.celsius:
        units = "C"
    else:
        units = "F"

    n = Nest(opts.user, opts.password, opts.serial, opts.index, units=units)
    n.login()
    n.get_status()

    cmd = args[0]

    if (cmd == "temp"):
        if len(args)<2:
            print "please specify a temperature"
            sys.exit(-1)
        n.set_temperature(int(args[1]))
    elif (cmd == "fan"):
        if len(args)<2:
            print "please specify a fan state of 'on' or 'auto'"
            sys.exit(-1)
        n.set_fan(args[1])
    elif (cmd == "show"):
        n.show_status()
        return res
    elif (cmd == "curtemp"):
        n.show_curtemp()
    elif (cmd == "curhumid"):
        print n.status["device"][n.serial]["current_humidity"]
    else:
        print "misunderstood command:", cmd
        print "do 'nest.py help' for help"

# if __name__=="__main__":
#     main()





In [173]:
username = 'zorroxinxsindi@gmail.com'
password = 'gdJide141@'
n = Nest(username, password, units = "F")
n.login()
n.get_status()
n.show_status()

$timestamp......................: 1481648378169
$version........................: -7418
alt_heat_delivery...............: forced-air
alt_heat_source.................: gas
alt_heat_x2_delivery............: forced-air
alt_heat_x2_source..............: gas
auto_away.......................: 0
auto_away_enable................: True
auto_away_learning..............: ready
auto_away_reset.................: False
auto_dehum_enabled..............: True
auto_dehum_state................: False
aux_heat_delivery...............: forced-air
aux_heat_source.................: electric
aux_lockout_leaf................: 10.0
available_locales...............: en_US,fr_CA,es_US,en_GB,fr_FR,nl_NL
away_temperature_high...........: 30.278
away_temperature_high_adjusted..: 30.278
away_temperature_high_enabled...: True
away_temperature_low............: 10.39597
away_temperature_low_adjusted...: 10.39597
away_temperature_low_enabled....: True
backplate_bsl_info..............: BSL
backplate_bsl_version..........

In [132]:
n.status

{u'buckets': {u'2925484': {u'$timestamp': 1480498030799,
   u'$version': -22704,
   u'buckets': [u'user.2925484',
    u'user_alert_dialog.2925484',
    u'message_center.2925484',
    u'user_settings.2925484',
    u'buckets.2925484',
    u'device.09AA01AC46150F2G',
    u'demand_response.09AA01AC46150F2G',
    u'device_alert_dialog.09AA01AC46150F2G',
    u'link.09AA01AC46150F2G',
    u'message.09AA01AC46150F2G',
    u'metadata.09AA01AC46150F2G',
    u'schedule.09AA01AC46150F2G',
    u'shared.09AA01AC46150F2G',
    u'track.09AA01AC46150F2G',
    u'tuneups.09AA01AC46150F2G',
    u'energy_latest.09AA01AC46150F2G',
    u'energy_weekly.09AA01AC46150F2G',
    u'tou.09AA01AC46150F2G',
    u'demand_charge.09AA01AC46150F2G',
    u'structure.37051b70-b997-11e5-9399-22000b6d823e',
    u'where.37051b70-b997-11e5-9399-22000b6d823e',
    u'utility.37051b70-b997-11e5-9399-22000b6d823e',
    u'structure_history.37051b70-b997-11e5-9399-22000b6d823e',
    u'trip.37051b70-b997-11e5-9399-22000b6d823e',
    

In [134]:
with open('data.json', 'w') as outfile:
    json.dump(n.status["device"]['09AA01AC46150F2G'], outfile)

In [112]:
print len(n.status)
dir(n.status)

25


['__class__',
 '__cmp__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'has_key',
 'items',
 'iteritems',
 'iterkeys',
 'itervalues',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values',
 'viewitems',
 'viewkeys',
 'viewvalues']

In [116]:
for item in n.status:
    print item, n.status[str(item)],'\n'

user_alert_dialog {u'2925484': {u'$version': 4635, u'dialog_id': u'confirm-pairing', u'dialog_data': u'', u'$timestamp': 1452649572044}} 

demand_response {u'09AA01AC46150F2G': {u'$version': -1, u'$timestamp': 1}} 

message_center {u'2925484': {u'$version': 1537, u'messages': [{u'parameters': [u'zorroxinxsindi@gmail.com'], u'read': True, u'timestamp': 1481546163, u'dismissed': True, u'priority': 2, u'thread_id': 0, u'key': u'mobile_device_health_check_fail', u'id': u'6d9fd500-c065-11e6-8826-22000acb84ae'}], u'$timestamp': 1481564517253}} 

message {u'09AA01AC46150F2G': {u'$version': 26113, u'$timestamp': 1477199673237}} 

trip {u'37051b70-b997-11e5-9399-22000b6d823e': {u'$version': 9300, u'trips': [], u'$timestamp': 1452649481991}} 

utility {u'37051b70-b997-11e5-9399-22000b6d823e': {u'$version': -1, u'$timestamp': 1}} 

user_settings {u'2925484': {u'lang': u'en_US', u'tos_current_version': 1434564000001, u'tos_accepted_version': 1434564000001, u'max_wwn_devices_per_structure': 10, u't

In [191]:
x = n.status["device"]['09AA01AC46150F2G']
#x = sorted(x)
#type(x)
# type(x.keys()[1])
#print x.keys(), ':',x.values(),'\n'

#@print x.keys()[10], ' ', type(x.keys()[10])

type(x['$timestamp'])

int

In [186]:
x = sorted(x)
type(x)


list

In [141]:
x = n.status["device"]['09AA01AC46150F2G']

for item in sorted(x):
    print item,':', x[str(item)]

$timestamp : 1481600788882
$version : 3965
alt_heat_delivery : forced-air
alt_heat_source : gas
alt_heat_x2_delivery : forced-air
alt_heat_x2_source : gas
auto_away_enable : True
auto_away_reset : False
auto_dehum_enabled : True
auto_dehum_state : False
aux_heat_delivery : forced-air
aux_heat_source : electric
aux_lockout_leaf : 10.0
available_locales : en_US,fr_CA,es_US,en_GB,fr_FR,nl_NL
away_temperature_high : 30.278
away_temperature_high_adjusted : 30.278
away_temperature_high_enabled : True
away_temperature_low : 10.39597
away_temperature_low_adjusted : 10.39597
away_temperature_low_enabled : True
backplate_bsl_info : BSL
backplate_bsl_version : 3.1
backplate_model : Backplate-5.3
backplate_mono_info : TFE (BP_D3) 1.2.9 (root@sticks.pao.corp.google.com) 2016-08-17 12:36:42
backplate_mono_version : 1.2.9
backplate_serial_number : 09DA01AC44150M4L
battery_level : 3.871
capability_level : 5.6
click_sound : on
compressor_lockout_leaf : -17.79999
cooling_delivery : unknown
cooling_sourc

In [148]:
y = x.keys()
type(y)

list

In [194]:
type(x)

dict

In [192]:
qmarks = ', '.join('?' * len(x))
print qmarks

?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?


TypeError: expected string or buffer

In [196]:
n.status["device"]['09AA01AC46150F2G']

{u'$timestamp': 1481648378169,
 u'$version': -7418,
 u'alt_heat_delivery': u'forced-air',
 u'alt_heat_source': u'gas',
 u'alt_heat_x2_delivery': u'forced-air',
 u'alt_heat_x2_source': u'gas',
 u'auto_away_enable': True,
 u'auto_away_reset': False,
 u'auto_dehum_enabled': True,
 u'auto_dehum_state': False,
 u'aux_heat_delivery': u'forced-air',
 u'aux_heat_source': u'electric',
 u'aux_lockout_leaf': 10.0,
 u'available_locales': u'en_US,fr_CA,es_US,en_GB,fr_FR,nl_NL',
 u'away_temperature_high': 30.278,
 u'away_temperature_high_adjusted': 30.278,
 u'away_temperature_high_enabled': True,
 u'away_temperature_low': 10.39597,
 u'away_temperature_low_adjusted': 10.39597,
 u'away_temperature_low_enabled': True,
 u'backplate_bsl_info': u'BSL',
 u'backplate_bsl_version': u'3.1',
 u'backplate_model': u'Backplate-5.3',
 u'backplate_mono_info': u'TFE (BP_D3) 1.2.9 (root@sticks.pao.corp.google.com) 2016-08-17 12:36:42',
 u'backplate_mono_version': u'1.2.9',
 u'backplate_serial_number': u'09DA01AC44150

In [197]:
len(x)

208

In [202]:
for item in x:
    print item, type(x[str(item)]),","

heat_pump_comp_threshold <type 'float'> ,
leaf_away_high <type 'float'> ,
schedules <type 'list'> ,
tou_icon <type 'bool'> ,
max_nighttime_preconditioning_seconds <type 'int'> ,
oob_temp_completed <type 'bool'> ,
postal_code <type 'unicode'> ,
heater_delivery <type 'unicode'> ,
country_code <type 'unicode'> ,
star_type <type 'unicode'> ,
heat_x3_source <type 'unicode'> ,
fan_timer_duration <type 'int'> ,
backplate_serial_number <type 'unicode'> ,
hvac_wires <type 'unicode'> ,
humidifier_type <type 'unicode'> ,
sunlight_correction_active <type 'bool'> ,
backplate_bsl_info <type 'unicode'> ,
temperature_lock <type 'bool'> ,
auto_away_reset <type 'bool'> ,
dual_fuel_breakpoint_override <type 'unicode'> ,
has_x3_heat <type 'bool'> ,
oob_wifi_completed <type 'bool'> ,
alt_heat_x2_delivery <type 'unicode'> ,
oob_startup_completed <type 'bool'> ,
maint_band_lower <type 'float'> ,
device_locale <type 'unicode'> ,
learning_time <type 'int'> ,
should_wake_on_approach <type 'bool'> ,
gear_thresho